In [1]:
import altair as alt
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
import os

path_base = Path(os.getcwd()).parents[1]
print(path_base)

pd.set_option('display.max_columns', 500)

/home/jkonovsky/development/notebooks/chicago_bears_nerd


In [12]:
week = 9

In [13]:
df = pd.read_csv(f"{path_base}/data/nfl_weekly_betting_summary.csv")
df = df.rename({"Unnamed: 5":"away"}, axis=1)
df[["result_ats","spread"]] = df.Spread.str.split(" ", expand=True)

df["result"] = df.Score.str.split(" ", expand=True)[0]
df[["fav_pts","opp_pts"]] = df.Score.str.split(" ", expand=True)[1].str.split("-", expand=True)

df["num_spread"] = pd.to_numeric(df.spread)
df['score_diff'] = pd.to_numeric(df.opp_pts) - pd.to_numeric(df.fav_pts)
team_map = {'Washington Commanders':"Was",
 'San Francisco 49ers':"SF",
 'Cleveland Browns':"Cle",
 'Green Bay Packers':"GB",
 'Indianapolis Colts':"Ind",
 'Minnesota Vikings':"Min",
 'Cincinnati Bengals':"Cin",
 'Baltimore Ravens':"Bal",
 'Tampa Bay Buccaneers':"TB",
 'Los Angeles Rams':"Rams",
 'Arizona Cardinals':"Az",
 'Buffalo Bills':"Buf",
 'Philadelphia Eagles':"Phi",
 'Los Angeles Chargers':"Charges",
'Chicago Bears':"Chi",
'Atlanta Falcons':"Atl",
'New England Patriots':"NE",
'New York Jets':"Jets",
'Jacksonville Jaguars':"Jac",
'Miami Dolphins':"Mia",
'New Orleans Saints':"No",
'New York Giants':"Giants",
'Pittsburgh Steelers':"Pit",
'Carolina Panthers':"Car",
'Seattle Seahawks':"Sea",
'Kansas City Chiefs':"KC",
'Dallas Cowboys':"Dal",
'Denver Broncos':"Den",
'Houston Texans': "Hou",
"Las Vegas Raiders":"LV",
"Tennessee Titans":"Ten",
"Detroit Lions":"Det"
}
df["fav_abr"] =  df.Favorite.map(team_map)
df["dog_abr"] = df.Underdog.map(team_map)
df["matchup"] = df.fav_abr + " vs " + df.dog_abr
df["fav_covered"] = (df.score_diff - df.num_spread) > 0
df["spread_v_result"] = df.score_diff - df.num_spread
df = df[df.week.eq(week)]

In [14]:
chart_title = f"Week {week} Score Differential Vs Game Spread"
x_cfgs = {"shorthand":"matchup", "title":"Fav - Dog"}
y_cfgs = {"shorthand":"score_diff", "title":"Score Differential (Dog pts - Fav pts) and Spread"}
# background etc.
primary_color = "#0B162A" 
# font etc.
secondary_color = "#c83803"
axisLeftKwargs = dict(labelColor=secondary_color, titleColor=secondary_color, labelFontSize=15, titleFontSize=17)
axisBottomKwargs = dict(labelColor=secondary_color, titleColor=secondary_color,labelFontSize=15, titleFontSize=17)
legendKwargs = dict(labelColor=secondary_color, titleColor=secondary_color, strokeColor='gray', fillColor=primary_color, padding=10, cornerRadius=10,orient='top-right', labelFontSize=15, titleFontSize=15)

df

chart = alt.Chart(df, width=800, height=500, title=chart_title)\
.mark_bar()\
.encode(
    x=alt.X(**x_cfgs,),
    y=alt.Y(**y_cfgs,
    ),
    tooltip=[
        #alt.Tooltip(shorthand="spread", title="Spread"),
        "Date",
        "Time (ET)",
        "Day",
        "Favorite",
        "Underdog",
        "Over/Under",
        "Spread",
        "Score",
        "score_diff",
        "spread_v_result"
    ],
    color=alt.condition(
        alt.datum.score_diff - alt.datum.num_spread > 0,
        alt.value(secondary_color),  # The positive color
        alt.value("white"),  # The negative color
        legend=True
    ),
)


tick = alt.Chart(df).mark_tick(
    color='red',
    width=50,
    thickness=5,
    size=40 * 0.9,  # controls width of tick.
).encode(
    x=alt.X(**x_cfgs,),
    y='num_spread'
)

plot = (chart + tick)\
.configure(background=primary_color)\
.configure_axis(gridColor="grey", gridDash=[6,4])\
.configure_axisLeft(**axisLeftKwargs)\
.configure_axisBottom(**axisBottomKwargs)\
.configure_title(color=secondary_color, fontSize=25)\
.configure_legend(**legendKwargs)

plot.save("chart_json/spread.json")
plot

In [24]:
df

,Unnamed: 0,Day,Date,Time (ET),Unnamed: 4,Favorite,Score,Spread,Unnamed: 8,Underdog,Over/Under,Notes,week,result_ats,spread,result,fav_pts,opp_pts,num_spread,score_diff,fav_abr,dog_abr,matchup,fav_covered,spread_v_result
108,0,Thu,"Oct 27, 2022",8:15,@,Tampa Bay Buccaneers,L 22-27,L -1.5,NaN,Baltimore Ravens,O 45.5,NaN,8,L,-1.5,L,22,27,-1.5,5,TB,Bal,TB vs Bal,True,6.5
109,1,Sun,"Oct 30, 2022",9:29,N,Jacksonville Jaguars,L 17-21,L -2,NaN,Denver Broncos,U 40.5,at London,8,L,-2,L,17,21,-2.0,4,Jac,Den,Jac vs Den,True,6.0
110,2,Sun,"Oct 30, 2022",1:00,@,Atlanta Falcons,W 37-34 (OT),L -4,NaN,Carolina Panthers,O 41,NaN,8,L,-4,W,37,34,-4.0,-3,Atl,Car,Atl vs Car,True,1.0
111,3,Sun,"Oct 30, 2022",1:00,@,Dallas Cowboys,W 49-29,W -10,NaN,Chicago Bears,O 42.5,NaN,8,W,-10,W,49,29,-10.0,-20,Dal,Chi,Dal vs Chi,False,-10.0
112,4,Sun,"Oct 30, 2022",1:00,NaN,Miami Dolphins,W 31-27,W -3.5,@,Detroit Lions,O 51.5,NaN,8,W,-3.5,W,31,27,-3.5,-4,Mia,Det,Mia vs Det,False,-0.5
113,5,Sun,"Oct 30, 2022",1:00,@,Minnesota Vikings,W 34-26,W -3.5,NaN,Arizona Cardinals,O 48.5,NaN,8,W,-3.5,W,34,26,-3.5,-8,Min,Az,Min vs Az,False,-4.5
114,6,Sun,"Oct 30, 2022",1:00,NaN,Las Vegas Raiders,L 0-24,L -1.5,@,New Orleans Saints,U 49.5,NaN,8,L,-1.5,L,0,24,-1.5,24,LV,No,LV vs No,True,25.5
115,7,Sun,"Oct 30, 2022",1:00,NaN,New England Patriots,W 22-17,W -2.5,@,New York Jets,U 40,NaN,8,W,-2.5,W,22,17,-2.5,-5,NE,Jets,NE vs Jets,False,-2.5
116,8,Sun,"Oct 30, 2022",1:00,@,Philadelphia Eagles,W 35-13,W -10.5,NaN,Pittsburgh Steelers,O 43,NaN,8,W,-10.5,W,35,13,-10.5,-22,Phi,Pit,Phi vs Pit,False,-11.5
117,9,Sun,"Oct 30, 2022",4:04,NaN,Tennessee Titans,W 17-10,W -3.5,@,Houston Texans,U 39,NaN,8,W,-3.5,W,17,10,-3.5,-7,Ten,Hou,Ten vs Hou,False,-3.5
